In [1]:
import gymnasium
from gymnasium import spaces
import numpy as np
import pandas as pd
import heapq
import matplotlib.pyplot as plt
from collections import defaultdict
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
import os
import random
import time

class RadarTaskSchedulerEnv(gymnasium.Env):
    def __init__(self, simulation_duration_minutes=60, render_mode=None):
        super(RadarTaskSchedulerEnv, self).__init__()

        self.simulation_duration_minutes = simulation_duration_minutes # Total simulation time for an episode
        self.current_simulation_time = 0.0 # Tracks the current time in the simulation

        # --- Radar Capabilities (Same as dataset generation) ---
        self.RADAR_CAPABILITIES = {
            'Radar1': ['Search', 'Detection', 'Classification'],
            'Radar2': ['Classification', 'Tracking', 'Detection'],
            'Radar3': ['Tracking', 'Search', 'Detection']
        }
        self.ALL_TASK_TYPES = ['Search', 'Detection', 'Tracking', 'Classification']

        # --- Stage Progression Logic (Same as dataset generation) ---
        self.MAX_STAGES_PER_TARGET_TYPE = {
            'Search': 1,        # 1 Search task is enough to potentially move to Detection
            'Detection': 4,     # Need 4 Detection tasks to move to Tracking
            'Tracking': 5,      # Need 5 Tracking tasks to move to Classification
            'Classification': 6 # Need 6 Classification tasks for 'Target Locked'
        }
        self.STAGE_ORDER = ['Search', 'Detection', 'Tracking', 'Classification', 'Locked']

        # --- Task Parameter Generation Ranges (Aligned with dataset generation) ---
        self.PRIORITY_MAP = {
            'Search': 1,
            'Detection': 2,
            'Tracking': 3,
            'Classification': 4
        }
        self.DEFAULT_PRIORITY_RANGE = (1, 5)

        self.AVG_TARGET_ARRIVAL_INTERVAL = 5.0 # Minutes between new target appearances

        self.DURATION_MAP = {
            'Search': (1, 3),
            'Detection': (2, 4),
            'Tracking': (3, 6),
            'Classification': (4, 8)
        }

        self.SLACK_FOR_DEADLINE_MAP = {
            'Search': (5, 15),
            'Detection': (4, 12),
            'Tracking': (3, 10),
            'Classification': (2, 8)
        }

        self.INITIAL_POWER_MAP = {
            'Search': (10, 20),
            'Detection': (20, 35),
            'Tracking': (30, 45),
            'Classification': (40, 60)
        }

        self.ADDITIONAL_MAX_POWER_RANGE = (0, 15)

        # --- Environment State ---
        self.event_queue = [] # (event_time, event_type, data) - for future target arrivals and task completions
        self.task_queue = [] # (priority, entry_count, task_dict) - tasks ready for scheduling (agent's choice)
        self.task_entry_count = 0 # Unique ID for task queue entries

        self.next_target_id = 1 # Counter for new target IDs

        # Key: Target_ID, Value: {'current_stage': 'Search', 'Search_count': 0, ..., 'last_processed_time': 0.0, 'is_locked': False}
        self.target_states = {}

        self.scheduled_tasks_log = [] # Log of all tasks processed in the current episode
        self.dropped_tasks_in_episode = 0 # Track dropped tasks

        self.current_task_for_observation = None # The task that the observation currently represents

        # Observation Space: [Duration, Deadline_Relative_to_Current_Time, Init_Power]
        # Max values are rough estimates, adapt if your ranges are different.
        # Deadline_Relative_to_Current_Time can be negative if already overdue.
        self.observation_space = spaces.Box(low=np.array([0, -self.simulation_duration_minutes, 0]),
                                             high=np.array([10, self.simulation_duration_minutes, 100]),
                                             shape=(3,), dtype=np.float32)

        # Action Space: 3 (Delay: 0, 1, 2) * 3 (Compress: 0, 1, 2) * 3 (Radar Choice: 0, 1, 2) = 27 actions
        self.action_space = spaces.Discrete(3 * 3 * 3)

        self.render_mode = render_mode
        self._last_info = {} # Stores info from the last step for internal use, including task_dropped
        self._last_reward = 0
        self._mode = "training" # New mode attribute: "training" or "inference"

    def set_mode(self, mode):
        """Sets the operating mode of the environment."""
        if mode not in ["training", "inference"]:
            raise ValueError("Mode must be 'training' or 'inference'")
        self._mode = mode

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)

        self.current_simulation_time = 0.0
        self.event_queue = []
        self.task_queue = []
        self.task_entry_count = 0
        self.next_target_id = 1
        self.target_states = {}
        self.scheduled_tasks_log = []
        self.dropped_tasks_in_episode = 0
        self.current_task_for_observation = None
        self._last_info = {} # Reset _last_info at the start of a new episode
        self._last_reward = 0

        # Only schedule initial target arrival in training mode
        if self._mode == "training":
            first_arrival_time = self.current_simulation_time + random.uniform(0.1, self.AVG_TARGET_ARRIVAL_INTERVAL * 1.5)
            heapq.heappush(self.event_queue, (first_arrival_time, 'TARGET_ARRIVAL', {'target_id': self.next_target_id}))
            self.next_target_id += 1

            # Advance simulation time until a task is ready or simulation ends
            self._advance_simulation_to_next_event()

        obs = self._get_observation()
        info = self._get_info()
        # Initialize Task_Dropped to False in info for the very first observation after reset
        info['Task_Dropped'] = False 
        return obs, info

    def _advance_simulation_to_next_event(self):
        """
        Processes events from the event queue until a task is available for the agent
        or the simulation duration is exceeded.
        """
        if self._mode == "inference":
            # In inference mode, we don't auto-advance or generate tasks.
            # Tasks are externally provided.
            return

        while self.event_queue and not self.task_queue and self.current_simulation_time < self.simulation_duration_minutes:
            event_time, event_type, event_data = heapq.heappop(self.event_queue)

            if event_time > self.simulation_duration_minutes:
                # Event is beyond simulation duration, don't process
                continue

            self.current_simulation_time = max(self.current_simulation_time, event_time)

            if event_type == 'TARGET_ARRIVAL':
                self._handle_target_arrival(event_data['target_id'])
                # Schedule next target arrival
                if self.current_simulation_time < self.simulation_duration_minutes:
                    next_arrival_time = self.current_simulation_time + random.uniform(
                        0.5 * self.AVG_TARGET_ARRIVAL_INTERVAL,
                        1.5 * self.AVG_TARGET_ARRIVAL_INTERVAL
                    )
                    if next_arrival_time < self.simulation_duration_minutes:
                        heapq.heappush(self.event_queue, (next_arrival_time, 'TARGET_ARRIVAL', {'target_id': self.next_target_id}))
                        self.next_target_id += 1

            elif event_type == 'TASK_COMPLETION':
                self._handle_task_completion_event(event_data)
        
        # Ensure current_simulation_time doesn't exceed duration if no more events
        if not self.event_queue and not self.task_queue and self._mode == "training":
            self.current_simulation_time = self.simulation_duration_minutes


    def _handle_target_arrival(self, target_id):
        """Initializes a new target and generates its first 'Search' task."""
        self.target_states[target_id] = {
            'current_stage': 'Search',
            'Search_count': 0,
            'Detection_count': 0,
            'Tracking_count': 0,
            'Classification_count': 0,
            'last_processed_time': self.current_simulation_time, # When this target became active
            'is_locked': False
        }
        # Generate the first search task for this new target
        self._generate_and_queue_task_for_target(target_id)

    def _handle_task_completion_event(self, completed_task_data):
        """
        Handles the internal completion of a task, which might trigger a new task
        for the same target.
        """
        target_id = completed_task_data['target_id']
        task_type = completed_task_data['task_type']
        
        target_state = self.target_states.get(target_id)
        if not target_state or target_state['is_locked']:
            return # Target already locked or doesn't exist (shouldn't happen for completion event)

        # Assuming completion means success for triggering next stage
        # (Reward logic already handles dropped/successful)
        # Check if this completion fulfills the requirements for the next stage
        
        if task_type in self.MAX_STAGES_PER_TARGET_TYPE: # Only stages that lead to progression
            required_count_for_next_stage = self.MAX_STAGES_PER_TARGET_TYPE[task_type]

            # If the current task type count in the state has reached the requirement AND
            # the task just completed was the last one for this stage before transitioning
            if target_state[f'{task_type}_count'] >= required_count_for_next_stage and \
               target_state['current_stage'] == task_type: # Ensure we are at this stage
                
                current_stage_index = self.STAGE_ORDER.index(task_type)
                if current_stage_index + 1 < len(self.STAGE_ORDER):
                    next_stage_type = self.STAGE_ORDER[current_stage_index + 1]
                    
                    if next_stage_type == 'Locked':
                        target_state['is_locked'] = True
                        target_state['current_stage'] = 'Locked' # Update state
                        return # Target is now locked, no new tasks
                    
                    # Transition the target to the next stage
                    target_state['current_stage'] = next_stage_type
                    # Do NOT reset the count for the new stage; it starts counting from 0 (implicitly)
                    # and the _generate_and_queue_task_for_target will increment it

        # Always try to generate a task for the target's current stage
        self._generate_and_queue_task_for_target(target_id)


    def _generate_and_queue_task_for_target(self, target_id):
        """
        Generates a task for the given target_id based on its current state and queues it.
        This is called for initial 'Search' tasks and subsequent tasks upon successful completion.
        """
        target_state = self.target_states.get(target_id)
        if not target_state or target_state['is_locked']:
            return # Target already locked

        task_type = target_state['current_stage']
        
        # Increment the count for the stage type we are about to generate
        target_state[f'{task_type}_count'] += 1
        current_task_stage_count = target_state[f'{task_type}_count']

        # Ensure we don't generate more tasks for a stage than specified in MAX_STAGES_PER_TARGET_TYPE,
        # unless it's a progression to the next stage which is handled by _handle_task_completion_event
        if task_type != 'Search' and current_task_stage_count > self.MAX_STAGES_PER_TARGET_TYPE[task_type]:
            return # No more tasks for this stage type

        # Generate task parameters
        request_time = self.current_simulation_time # Task is requested now
        
        duration_range = self.DURATION_MAP.get(task_type, (2, 8))
        duration = random.randint(duration_range[0], duration_range[1])

        slack_range = self.SLACK_FOR_DEADLINE_MAP.get(task_type, (3, 10))
        slack_for_deadline = random.randint(slack_range[0], slack_range[1])
        deadline = round(request_time + duration + slack_for_deadline, 2)

        initial_power_range = self.INITIAL_POWER_MAP.get(task_type, (10, 50))
        initial_power = random.randint(initial_power_range[0], initial_power_range[1])
        max_power = initial_power + random.randint(self.ADDITIONAL_MAX_POWER_RANGE[0], self.ADDITIONAL_MAX_POWER_RANGE[1])

        max_delay = round(deadline - request_time - duration, 2)
        if max_delay < 0: max_delay = 0.0

        eligible_radars = [r for r, caps in self.RADAR_CAPABILITIES.items() if task_type in caps]
        radar_type = random.choice(eligible_radars) if eligible_radars else "AnyRadar"

        priority = self.PRIORITY_MAP.get(task_type, random.randint(self.DEFAULT_PRIORITY_RANGE[0], self.DEFAULT_PRIORITY_RANGE[1]))

        new_task = {
            'Target_ID': target_id,
            'Task_ID': f"{target_id}_{task_type[0]}{current_task_stage_count}",
            'Task_Type': task_type,
            'Radar_Type': radar_type,
            'Priority': priority,
            'Request_Time': request_time,
            'Deadline': deadline,
            'Duration': duration,
            'Init_Power': initial_power,
            'Max_Power': max_power,
            'Max_Delay': max_delay,
            'Stage_Count': current_task_stage_count
        }
        
        # Push to the task queue for the agent to observe and act upon
        negative_priority = -new_task['Priority']
        heapq.heappush(self.task_queue, (negative_priority, self.task_entry_count, new_task))
        self.task_entry_count += 1


    def step(self, action_index):
        reward = 0.0
        truncated = False
        done = False

        # Initialize current_step_task_dropped for the current step. 
        # It will be updated if a task is processed.
        current_step_task_dropped = False 

        if not self.task_queue:
            # If no tasks are available in training mode, advance simulation
            if self._mode == "training":
                self._advance_simulation_to_next_event()
            
            if not self.task_queue: # Still no tasks after advancing time or in inference mode
                if self.current_simulation_time >= self.simulation_duration_minutes and self._mode == "training":
                    done = True
                    self._last_info['message'] = "Simulation duration reached with no tasks."
                else:
                    # Penalty for idle time/no task to process
                    reward -= 0.1 
                    self._last_info['message'] = "No tasks available for decision."
                
                # If no task is processed, current_step_task_dropped for this step is False
                self._last_info['Task_Dropped'] = current_step_task_dropped # Ensure this is always set
                self._last_info['reward_from_last_step'] = reward # Update the reward for this 'idle' step
                
                obs = self._get_observation()
                info = self._get_info() 
                return obs, reward, done, truncated, info

        # If a task is now available
        _priority, _entry_count, task_to_process = heapq.heappop(self.task_queue)
        
        # Action Decoding
        delay_action = action_index // 9 # 0, 1, 2 (corresponding to 0, 1, 2 minutes delay)
        compress_action = (action_index % 9) // 3 # 0, 1, 2 (corresponding to 0%, 10%, 20% compression)
        radar_choice_action = action_index % 3 # 0, 1, 2 (for 3 radars)

        # Calculate actual TNS and Pn based on actions
        calculated_tns = task_to_process['Request_Time'] + delay_action
        
        # Assuming compress_action 0 = no additional power, 1 = 50% of additional_max_power, 2 = 100%
        additional_power_from_compression = (task_to_process['Max_Power'] - task_to_process['Init_Power']) * (compress_action / 2.0)
        calculated_Pn = task_to_process['Init_Power'] + additional_power_from_compression

        # --- PROPOSED CHANGE ---
        # Define how much power each compression level *adds* to the initial power.
        # These values should be chosen such that they can potentially push calculated_Pn > Max_Power.
        # You can tune these values based on your simulation's requirements.
        # Example: No compression (0), Medium compression (1), High compression (2)
        if compress_action == 0:
            power_increase_from_compression = 0.0
        elif compress_action == 1:
            power_increase_from_compression = 0.15 * task_to_process['Init_Power'] # Example: 15% of initial power
        elif compress_action == 2:
            power_increase_from_compression = 0.30 * task_to_process['Init_Power'] # Example: 30% of initial power
        else: # Should not happen with current action space
            power_increase_from_compression = 0.0

        calculated_Pn = task_to_process['Init_Power'] + power_increase_from_compression

        # Check for task drop conditions
        if (calculated_tns + task_to_process['Duration']) > task_to_process['Deadline']:
            current_step_task_dropped = True
        if calculated_Pn > task_to_process['Max_Power']:
            current_step_task_dropped = True
        
        # --- Reward Calculation ---
        reward = 0.0 # Re-initialize for this task's specific reward calculation

        if not current_step_task_dropped:
            reward += 1.0 # Base reward for successful completion

            # Time-based reward
            time_remaining = task_to_process['Deadline'] - calculated_tns
            if time_remaining >= 0:
                reward += time_remaining * 0.05 # Bonus for time remaining
            else: # Should be caught by task_dropped, but as a safeguard
                reward -= 0.5

            # Power-based reward
            power_margin = task_to_process['Max_Power'] - calculated_Pn
            if power_margin >= 0:
                reward += power_margin * 0.01 # Bonus for power efficiency
            else:
                reward -= 0.5

            # Incentive for minimal delay/compression if successful
            if delay_action == 0:
                reward += 0.1
            if compress_action == 0:
                reward += 0.1
            
            # Special reward for 'locking' a target (reaching Classification_6 and successfully completing)
            target_id = task_to_process['Target_ID']
            task_type = task_to_process['Task_Type']
            stage_count = task_to_process['Stage_Count']

            # Update target state *before* checking for locked, so the completion event can proceed
            # If the target doesn't exist, initialize it (needed for inference mode where targets don't 'arrive' via events)
            if target_id not in self.target_states:
                self.target_states[target_id] = {
                    'current_stage': 'Search',
                    'Search_count': 0, 'Detection_count': 0,
                    'Tracking_count': 0, 'Classification_count': 0,
                    'last_processed_time': self.current_simulation_time,
                    'is_locked': False
                }
            
            # Update the count for the current task type in the target state
            self.target_states[target_id][f'{task_type}_count'] = stage_count # This reflects the count AFTER this task

            if task_type == 'Classification' and stage_count == self.MAX_STAGES_PER_TARGET_TYPE['Classification']:
                # This task completes the 'Classification' sequence for this target
                reward += 10.0 # Significant bonus for locking a target
                self.target_states[target_id]['is_locked'] = True
                self.target_states[target_id]['current_stage'] = 'Locked' # Update state

        else:
            reward -= 5.0 # Larger penalty for dropping a task
            self.dropped_tasks_in_episode += 1

        # Log the processed task (this is distinct from the info dictionary returned by step)
        self.scheduled_tasks_log.append({
            'Current_Sim_Time': self.current_simulation_time, # When the action was taken
            'Target_ID': task_to_process['Target_ID'],
            'Task_ID': task_to_process['Task_ID'],
            'Task_Type': task_to_process['Task_Type'],
            'Priority': task_to_process['Priority'],
            'Request_Time': task_to_process['Request_Time'],
            'Original_Deadline': task_to_process['Deadline'],
            'Original_Duration': task_to_process['Duration'],
            'Original_Init_Power': task_to_process['Init_Power'],
            'Original_Max_Power': task_to_process['Max_Power'],
            'Stage_Count': task_to_process['Stage_Count'],
            'Action_Delay': delay_action,
            'Action_Compress': compress_action,
            'Action_Radar': radar_choice_action,
            'Calculated_TNS': calculated_tns,
            'Calculated_Pn': calculated_Pn,
            'Task_Dropped': current_step_task_dropped, # Use the current step's task_dropped status
            'Reward_Earned': reward
        })

        # Calculate completion_event_time first, then use it in the conditional
        completion_event_time = calculated_tns + task_to_process['Duration']

        # Schedule a 'TASK_COMPLETION' event for this task, which will happen at `calculated_tns + Duration`
        # This event will then trigger the next task generation for the target if applicable
        if not current_step_task_dropped and self._mode == "training" and completion_event_time < self.simulation_duration_minutes:
            heapq.heappush(self.event_queue, (completion_event_time, 'TASK_COMPLETION', {
                'target_id': task_to_process['Target_ID'],
                'task_type': task_to_process['Task_Type'],
                'stage_count': task_to_process['Stage_Count'] # Pass current stage count to completion handler
            }))
        elif self._mode == "inference" and not current_step_task_dropped:
            # In inference, we directly trigger the next stage logic if successful
            # The 'current_simulation_time' here just represents the time this task was processed
            self.current_simulation_time = completion_event_time
            self._handle_task_completion_event({
                'target_id': task_to_process['Target_ID'],
                'task_type': task_to_process['Task_Type'],
                'stage_count': task_to_process['Stage_Count']
            })

        # Advance simulation time after processing an action (only in training mode).
        if self._mode == "training":
            self._advance_simulation_to_next_event()

        # Check for done condition:
        if self._mode == "training":
            if self.current_simulation_time >= self.simulation_duration_minutes:
                done = True
            elif not self.task_queue and not self.event_queue:
                done = True # Done if no more tasks are coming AND simulation time has passed
        else: # Inference mode - done will be controlled externally by the loop feeding tasks
            done = False # Environment itself is not 'done' until external loop signals

        # --- Update _last_info for the current step's outcome ---
        self._last_info['Task_Dropped'] = current_step_task_dropped # Ensure this is explicitly set
        self._last_info['reward_from_last_step'] = reward # Update for the current step's actual reward

        info = self._get_info() # Calls _get_info to compile common info
        
        # Ensure _last_reward is consistent with the reward returned by this step
        self._last_reward = reward 

        obs = self._get_observation()
        return obs, reward, done, truncated, info

    def _get_observation(self):
        """
        Returns an observation based on the highest priority task currently in the queue.
        If no tasks, returns a zero observation.
        """
        if self.task_queue:
            # Peek at the top task (highest priority)
            _priority, _entry_count, task_dict = self.task_queue[0]
            self.current_task_for_observation = task_dict

            # Observation: [Duration, Deadline_Relative_to_Current_Time, Init_Power]
            # Deadline_Relative_to_Current_Time can be negative if it's already overdue.
            deadline_relative = task_dict['Deadline'] - self.current_simulation_time
            
            return np.array([task_dict['Duration'], deadline_relative, task_dict['Init_Power']], dtype=np.float32)
        else:
            self.current_task_for_observation = None
            # Return a default observation if no tasks are available
            return np.array([0, -self.simulation_duration_minutes, 0], dtype=np.float32)

    def _get_info(self):
        """Provides additional debugging/logging information."""
        info = {
            "current_simulation_time": self.current_simulation_time,
            "tasks_in_queue": len(self.task_queue),
            "events_in_queue": len(self.event_queue),
            "dropped_tasks_in_episode": self.dropped_tasks_in_episode,
            "completed_tasks_count": len(self.scheduled_tasks_log) - self.dropped_tasks_in_episode,
            "total_tasks_processed_this_step": len(self.scheduled_tasks_log),
            # Access reward_from_last_step from _last_info. It's updated in step().
            "reward_from_last_step": self._last_info.get('reward_from_last_step', 0.0), 
            "targets_locked": sum(1 for ts in self.target_states.values() if ts['is_locked'])
        }
        if self.current_task_for_observation:
            info["observing_task_id"] = self.current_task_for_observation['Task_ID']
            info["observing_task_type"] = self.current_task_for_observation['Task_Type']
            info["observing_target_id"] = self.current_task_for_observation['Target_ID']
            info["observing_request_time"] = self.current_task_for_observation['Request_Time']
        else:
            info["observing_task_id"] = "N/A"
        
        # Add 'Task_Dropped' from _last_info to the returned info dictionary
        info['Task_Dropped'] = self._last_info.get('Task_Dropped', False)
        
        return info


    def close(self):
        pass

# --- Episode Logger Callback (Adjusted for new info structure) ---
class EpisodeLoggerCallback(BaseCallback):
    def __init__(self, output_file, total_episodes_target, verbose=0):
        super(EpisodeLoggerCallback, self).__init__(verbose)
        self.output_file = output_file
        self.total_episodes_target = total_episodes_target
        self.episode_num = 0
        self.reward_log = []
        self.task_drop_log = []
        self.completed_tasks_log = []
        self.targets_locked_log = []

    def _on_step(self) -> bool:
        if self.locals['dones'][0]:
            self.episode_num += 1
            
            last_step_info = self.locals['infos'][0]
            
            # Monitor wrapper adds 'episode' key to info dict
            if 'episode' in last_step_info:
                episode_stats = last_step_info['episode']
                total_reward_episode = episode_stats['r']
                
                # These metrics are now directly from the environment's info at the end of the episode
                task_drops_episode = last_step_info.get('dropped_tasks_in_episode', 0)
                completed_tasks_episode = last_step_info.get('completed_tasks_count', 0)
                targets_locked_episode = last_step_info.get('targets_locked', 0)
                
                self.reward_log.append(total_reward_episode)
                self.task_drop_log.append(task_drops_episode)
                self.completed_tasks_log.append(completed_tasks_episode)
                self.targets_locked_log.append(targets_locked_episode)

                # Access exploration_rate directly from the model, handling its absence gracefully
                epsilon = getattr(self.model, 'exploration_rate', 'N/A')

                log_message = (
                    f"Episode {self.episode_num}/{self.total_episodes_target} "
                    f"— Total Reward: {total_reward_episode:.2f} "
                    f"— Tasks Dropped: {task_drops_episode} "
                    f"— Tasks Completed: {completed_tasks_episode} "
                    f"— Targets Locked: {targets_locked_episode} "
                    f"— Sim Time: {last_step_info.get('current_simulation_time', 'N/A'):.2f} min "
                    f"— Epsilon (approx): {epsilon:.3f}" if isinstance(epsilon, float) else f"— Epsilon (approx): {epsilon}"
                )
                print(log_message)
                self.output_file.write(log_message + "\n")
        return True

    def _on_rollout_end(self) -> None:
        pass

    def _on_training_end(self) -> None:
        pass

# --- Main Training and Evaluation Script ---
if __name__ == "__main__":
    SIM_DURATION = 120 # minutes per episode (e.g., 2 hours)
    MODEL_PATH = "dqn_radar_scheduler_dynamic_event_driven_model"
    OUTPUT_LOG_FILE = "dqn_learning_results_dynamic_event_driven.txt"
    EVAL_LOG_FILE = "dqn_inference_results.txt"

    # --- Training Phase ---
    print("\n--- Starting Training Phase ---")
    base_env = RadarTaskSchedulerEnv(simulation_duration_minutes=SIM_DURATION, render_mode=None)
    env = Monitor(base_env) # Monitor wraps the environment to log episode statistics

    dqn_hyperparams = {
        "learning_rate": 0.0005,
        "buffer_size": 50000, # Increased buffer size for continuous task flow
        "learning_starts": 2000, # Increased learning starts
        "batch_size": 64,
        "gamma": 0.99,
        "train_freq": (4, "step"),
        "gradient_steps": 1,
        "target_update_interval": 200,
        "exploration_fraction": 0.3, # Slightly longer exploration
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.05,
        "max_grad_norm": 10,
        "seed": 42
    }

    model = DQN("MlpPolicy", env, verbose=0, **dqn_hyperparams, device="auto")

    episodes_to_train = 100
    # A step roughly corresponds to processing one task or advancing time to the next event.
    # Total timesteps should be enough to cover many episodes and a significant number of tasks.
    # A rough estimate for total timesteps could be (episodes * avg_tasks_per_episode * avg_steps_per_task).
    # For now, a generous estimate:
    total_timesteps_for_training = episodes_to_train * 500 # Assuming ~500 steps/tasks per episode

    with open(OUTPUT_LOG_FILE, "w") as f_out:
        f_out.write("--- DQN Training Log (Dynamic Event-Driven Tasks) ---\n\n")
        print("Starting DQN training with dynamic event-driven task generation...")
        f_out.write("Starting DQN training with dynamic event-driven task generation...\n\n")

        episode_logger_callback = EpisodeLoggerCallback(
            output_file=f_out,
            total_episodes_target=episodes_to_train
        )

        model.learn(total_timesteps=total_timesteps_for_training, callback=episode_logger_callback)

        print("\nTraining complete!")
        f_out.write("\nTraining complete!\n")

        model.save(MODEL_PATH)
        print(f"Final model saved to {MODEL_PATH}")
        f_out.write(f"Final model saved to {MODEL_PATH}\n")

        # --- Plotting Results (Training) ---
        reward_log = episode_logger_callback.reward_log
        task_drop_log = episode_logger_callback.task_drop_log
        completed_tasks_log = episode_logger_callback.completed_tasks_log
        targets_locked_log = episode_logger_callback.targets_locked_log

        window_size = 50 # For moving average

        # Plot Rewards
        if reward_log:
            plt.figure(figsize=(14,7))
            plt.plot(reward_log, label='Episode Rewards', color='orange', alpha=0.7, linewidth=1)
            if len(reward_log) >= window_size:
                moving_avg = np.convolve(reward_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg = range(window_size - 1, len(reward_log))
                plt.plot(x_moving_avg, moving_avg, label=f'Moving Average ({window_size} episodes)', color='blue', linewidth=2)
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Total Reward', fontsize=12)
            plt.title('Total Reward over Episodes (DQN - Dynamic Event-Driven)', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("dqn_reward_plot_dynamic_event_driven.png")
            plt.close()
            print(f"Reward plot saved as dqn_reward_plot_dynamic_event_driven.png")
            f_out.write(f"Reward plot saved as dqn_reward_plot_dynamic_event_driven.png\n")
        else:
            print("\nReward log is empty. No reward plot will be generated.")
            f_out.write("\nReward log is empty. No reward plot will be generated.\n")

        # Plot Targets Locked
        if targets_locked_log:
            plt.figure(figsize=(14,7))
            plt.plot(targets_locked_log, label='Targets Locked per Episode', color='green', alpha=0.7, linewidth=1)
            if len(targets_locked_log) >= window_size:
                moving_avg_locked = np.convolve(targets_locked_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg_locked = range(window_size - 1, len(targets_locked_log))
                plt.plot(x_moving_avg_locked, moving_avg_locked, label=f'Moving Average Targets Locked ({window_size} episodes)', color='purple', linewidth=2)
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Number of Targets Locked', fontsize=12)
            plt.title('Targets Locked over Episodes (DQN - Dynamic Event-Driven)', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("dqn_targets_locked_plot_dynamic_event_driven.png")
            plt.close()
            print(f"Targets locked plot saved as dqn_targets_locked_plot_dynamic_event_driven.png")
            f_out.write(f"Targets locked plot saved as dqn_targets_locked_plot_dynamic_event_driven.png\n")
        else:
            print("\nTargets locked log is empty. No targets locked plot will be generated.")
            f_out.write("\nTargets locked log is empty. No targets locked plot will be generated.\n")

    # --- Inference Phase: One Task at a Time ---
    print("\n--- Starting Inference Phase (One Task at a Time) ---")
    try:
        # Load the trained model
        loaded_model = DQN.load(MODEL_PATH)
        print(f"Model loaded successfully from {MODEL_PATH}")

        # Create a new environment for inference mode
        inference_env = RadarTaskSchedulerEnv(simulation_duration_minutes=SIM_DURATION) # Duration might be irrelevant here if tasks are fed
        inference_env.set_mode("inference") # Set the environment to inference mode
        # Reset the environment for inference (clears queues but doesn't auto-generate tasks)
        obs, info = inference_env.reset() 

        # Create a list of example tasks to process
        # In a real scenario, this would come from a file, a sensor, or a real-time queue.
        # Here, we generate a few specific tasks and then some random ones to demonstrate.
        example_tasks_to_process = [
            # Example 1: Search task for a new target
            {
                'Target_ID': 101, 'Task_ID': '101_S1', 'Task_Type': 'Search', 'Radar_Type': 'Radar1',
                'Priority': 1, 'Request_Time': 0.0, 'Deadline': 10.0, 'Duration': 2,
                'Init_Power': 15, 'Max_Power': 30, 'Max_Delay': 8.0, 'Stage_Count': 1
            },
            # Example 2: Detection task for the same target (assuming S1 was completed)
            {
                'Target_ID': 101, 'Task_ID': '101_D1', 'Task_Type': 'Detection', 'Radar_Type': 'Radar2',
                'Priority': 2, 'Request_Time': 2.0, 'Deadline': 15.0, 'Duration': 3,
                'Init_Power': 25, 'Max_Power': 40, 'Max_Delay': 10.0, 'Stage_Count': 1
            },
             # Example 3: Another detection task for the same target
            {
                'Target_ID': 101, 'Task_ID': '101_D2', 'Task_Type': 'Detection', 'Radar_Type': 'Radar2',
                'Priority': 2, 'Request_Time': 5.0, 'Deadline': 20.0, 'Duration': 3,
                'Init_Power': 25, 'Max_Power': 40, 'Max_Delay': 12.0, 'Stage_Count': 2 # Manually increment stage count
            },
            # Example 4: A new target's search task
            {
                'Target_ID': 102, 'Task_ID': '102_S1', 'Task_Type': 'Search', 'Radar_Type': 'Radar3',
                'Priority': 1, 'Request_Time': 1.0, 'Deadline': 12.0, 'Duration': 2,
                'Init_Power': 12, 'Max_Power': 25, 'Max_Delay': 9.0, 'Stage_Count': 1
            },
        ]
        
        # Also add some dynamically generated tasks to simulate more varied arrivals
        for i in range(5):
            target_id_rand = 200 + i
            task_type_rand = random.choice(['Search', 'Detection', 'Tracking', 'Classification'])
            duration_range_rand = inference_env.DURATION_MAP.get(task_type_rand, (2, 8))
            duration_rand = random.randint(duration_range_rand[0], duration_range_rand[1])
            slack_range_rand = inference_env.SLACK_FOR_DEADLINE_MAP.get(task_type_rand, (3, 10))
            slack_for_deadline_rand = random.randint(slack_range_rand[0], slack_range_rand[1])
            request_time_rand = inference_env.current_simulation_time + random.uniform(1, 5) # Simulate slight time progression
            deadline_rand = round(request_time_rand + duration_rand + slack_for_deadline_rand, 2)
            initial_power_range_rand = inference_env.INITIAL_POWER_MAP.get(task_type_rand, (10, 50))
            initial_power_rand = random.randint(initial_power_range_rand[0], initial_power_range_rand[1])
            max_power_rand = initial_power_rand + random.randint(inference_env.ADDITIONAL_MAX_POWER_RANGE[0], inference_env.ADDITIONAL_MAX_POWER_RANGE[1])
            max_delay_rand = round(deadline_rand - request_time_rand - duration_rand, 2)
            if max_delay_rand < 0: max_delay_rand = 0.0

            eligible_radars_rand = [r for r, caps in inference_env.RADAR_CAPABILITIES.items() if task_type_rand in caps]
            radar_type_rand = random.choice(eligible_radars_rand) if eligible_radars_rand else "AnyRadar"
            priority_rand = inference_env.PRIORITY_MAP.get(task_type_rand, random.randint(inference_env.DEFAULT_PRIORITY_RANGE[0], inference_env.DEFAULT_PRIORITY_RANGE[1]))

            stage_count_rand = 1 # Default to 1 for newly generated tasks
            example_tasks_to_process.append({
                'Target_ID': target_id_rand, 'Task_ID': f"{target_id_rand}_{task_type_rand[0]}X", 'Task_Type': task_type_rand,
                'Radar_Type': radar_type_rand, 'Priority': priority_rand, 'Request_Time': request_time_rand,
                'Deadline': deadline_rand, 'Duration': duration_rand, 'Init_Power': initial_power_rand,
                'Max_Power': max_power_rand, 'Max_Delay': max_delay_rand, 'Stage_Count': stage_count_rand 
            })


        total_inference_tasks = len(example_tasks_to_process)
        tasks_processed_count = 0
        inference_results_log = []
        total_inference_reward = 0
        total_targets_locked_inference = 0

        with open(EVAL_LOG_FILE, "w") as f_eval:
            f_eval.write("--- DQN Inference Log (One Task at a Time) ---\n\n")

            for i, task_data in enumerate(example_tasks_to_process):
                tasks_processed_count += 1
                print(f"\nProcessing Inference Task {tasks_processed_count}/{total_inference_tasks}: {task_data['Task_ID']} (Type: {task_data['Task_Type']})")
                f_eval.write(f"\nProcessing Inference Task {tasks_processed_count}/{total_inference_tasks}: {task_data['Task_ID']} (Type: {task_data['Task_Type']})\n")

                # Manually push the current task to the environment's task_queue
                inference_env.task_queue = [] # Clear any previous task if it exists
                inference_env.current_simulation_time = task_data['Request_Time'] # Set time to task request time for observation

                # Manually handle target state for tasks in inference mode 
                if task_data['Target_ID'] not in inference_env.target_states:
                    inference_env.target_states[task_data['Target_ID']] = {
                        'current_stage': task_data['Task_Type'], # Assume this is the current stage
                        'Search_count': 0, 'Detection_count': 0,
                        'Tracking_count': 0, 'Classification_count': 0,
                        'last_processed_time': task_data['Request_Time'],
                        'is_locked': False
                    }
                # For inference, the `Stage_Count` in the task_data represents the count *this* task is.
                # The environment's `step` method will use this.

                neg_priority = -task_data['Priority']
                heapq.heappush(inference_env.task_queue, (neg_priority, inference_env.task_entry_count, task_data))
                inference_env.task_entry_count += 1

                # Get observation for the current task
                current_obs = inference_env._get_observation()
                
                # Model predicts action
                action_index, _states = loaded_model.predict(current_obs, deterministic=True)
                
                # Apply action and get reward/info
                obs, reward, done, truncated, info = inference_env.step(action_index)

                log_line = (
                    f"  Action: Delay={action_index // 9}, Compress={(action_index % 9) // 3}, Radar={action_index % 3} "
                    f"— Dropped: {info['Task_Dropped']} " # Access Task_Dropped from the returned info dict
                    f"— Reward: {reward:.2f} "
                    f"— Sim Time (post-process): {info['current_simulation_time']:.2f} min "
                    f"— Target {info['observing_target_id']} Stage: {inference_env.target_states.get(info['observing_target_id'], {}).get('current_stage', 'N/A')}"
                )
                print(log_line)
                f_eval.write(log_line + "\n")

                inference_results_log.append({
                    'Task_ID': task_data['Task_ID'],
                    'Task_Type': task_data['Task_Type'],
                    'Target_ID': task_data['Target_ID'],
                    'Task_Dropped': info['Task_Dropped'], # Access Task_Dropped from the returned info dict
                    'Reward': reward,
                    'Final_Stage': inference_env.target_states.get(task_data['Target_ID'], {}).get('current_stage', 'N/A'),
                    'Is_Locked': inference_env.target_states.get(task_data['Target_ID'], {}).get('is_locked', False)
                })
                total_inference_reward += reward

                if inference_env.target_states.get(task_data['Target_ID'], {}).get('is_locked', False):
                    print(f"  !!! Target {task_data['Target_ID']} LOCKED !!!")
                    f_eval.write(f"  !!! Target {task_data['Target_ID']} LOCKED !!!\n")
                    total_targets_locked_inference += 1

                # Critical for one-task-at-a-time: clear the task queue for the next external task
                inference_env.task_queue = []

            print(f"\n--- Inference Complete ---")
            print(f"Total Tasks Processed: {tasks_processed_count}")
            print(f"Total Inference Reward: {total_inference_reward:.2f}")
            print(f"Total Targets Locked: {total_targets_locked_inference}")
            f_eval.write(f"\n--- Inference Complete ---\n")
            f_eval.write(f"Total Tasks Processed: {tasks_processed_count}\n")
            f_eval.write(f"Total Inference Reward: {total_inference_reward:.2f}\n")
            f_eval.write(f"Total Targets Locked: {total_targets_locked_inference}\n")

    except Exception as e:
        print(f"An error occurred during inference: {e}")
        if os.path.exists(EVAL_LOG_FILE):
            with open(EVAL_LOG_FILE, "a") as f_eval:
                f_eval.write(f"\nAn error occurred during inference: {e}\n")


--- Starting Training Phase ---
Starting DQN training with dynamic event-driven task generation...
Episode 1/100 — Total Reward: -13.38 — Tasks Dropped: 21 — Tasks Completed: 56 — Targets Locked: 0 — Sim Time: 120.00 min — Epsilon (approx): 0.995
Episode 2/100 — Total Reward: -39.20 — Tasks Dropped: 23 — Tasks Completed: 46 — Targets Locked: 0 — Sim Time: 120.00 min — Epsilon (approx): 0.991
Episode 3/100 — Total Reward: -17.88 — Tasks Dropped: 20 — Tasks Completed: 49 — Targets Locked: 0 — Sim Time: 120.00 min — Epsilon (approx): 0.986
Episode 4/100 — Total Reward: 11.39 — Tasks Dropped: 24 — Tasks Completed: 79 — Targets Locked: 0 — Sim Time: 120.00 min — Epsilon (approx): 0.980
Episode 5/100 — Total Reward: -37.43 — Tasks Dropped: 19 — Tasks Completed: 34 — Targets Locked: 0 — Sim Time: 120.00 min — Epsilon (approx): 0.977
Episode 6/100 — Total Reward: -27.58 — Tasks Dropped: 24 — Tasks Completed: 55 — Targets Locked: 0 — Sim Time: 120.00 min — Epsilon (approx): 0.972
Episode 7/100

In [3]:
# --- Inference Phase ---
print("\n--- Starting Inference Phase ---")

# Load the trained model
try:
    inference_model = DQN.load(MODEL_PATH)
    print(f"Model loaded successfully from {MODEL_PATH}")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Instantiate the environment for inference mode
inference_env = RadarTaskSchedulerEnv(simulation_duration_minutes=SIM_DURATION)
inference_env.set_mode("inference") # Set the environment to inference mode

# Load the inference tasks from the CSV
try:
    inference_tasks_df = pd.read_csv("dynamic_radar_tasks_full_progression.csv")
    # Ensure correct data types, especially for numerical columns
    numerical_cols = ['Priority', 'Request_Time', 'Deadline', 'Duration', 'Init_Power', 'Max_Power', 'Max_Delay', 'Stage_Count']
    for col in numerical_cols:
        if col in inference_tasks_df.columns:
            inference_tasks_df[col] = pd.to_numeric(inference_tasks_df[col], errors='coerce')
    inference_tasks_df.dropna(subset=numerical_cols, inplace=True)

    print(f"Loaded {len(inference_tasks_df)} tasks for inference.")
except FileNotFoundError:
    print("Error: dynamic_radar_tasks_full_progression.csv not found. Please ensure it's in the same directory.")
    exit()
except Exception as e:
    print(f"Error loading or processing CSV: {e}")
    exit()

# Prepare a log for inference results
inference_results_log = []
total_inference_reward = 0
inference_dropped_tasks = 0
inference_locked_targets = 0

# Reset the environment for the first inference run
obs, info = inference_env.reset()

# Process tasks one by one in inference mode
# ... inside your inference loop ...
for index, task_row in inference_tasks_df.iterrows():
    task_dict = task_row.to_dict()

    # ... (your existing code for setting current_simulation_time and pushing task) ...

    obs = inference_env._get_observation()
    action_index, _states = inference_model.predict(obs, deterministic=True)
    delay_action = action_index // 9 # 0, 1, 2
    compress_action = (action_index % 9) // 3 # 0, 1, 2
    radar_choice_action = action_index % 3 # 0, 1, 2


    # Log inference results for each task
    inference_results_log.append({
        'Task_Index': index,
        'Task_ID': task_dict['Task_ID'],
        'Predicted_Action_Delay': delay_action,  # Use decoded action
        'Predicted_Action_Compress': compress_action, # Use decoded action
        'Predicted_Action_Radar': radar_choice_action, # Use decoded action
        'Task_Dropped': info['Task_Dropped'],
        'Reward_Earned': reward,
        'Current_Sim_Time_After_Processing': info['current_simulation_time'],
        'Targets_Locked_After_Task': info['targets_locked']
    })
    total_inference_reward += reward
    if info['Task_Dropped']:
        inference_dropped_tasks += 1

    # Update locked targets for logging (info['targets_locked'] is cumulative)
    inference_locked_targets = info['targets_locked']

    # For ongoing progress during inference
    if (index + 1) % 100 == 0 or (index + 1) == len(inference_tasks_df):
        print(f"Processed {index + 1}/{len(inference_tasks_df)} inference tasks. Last Reward: {reward:.2f}")

print("\n--- Inference Complete ---")
print(f"Total Inference Tasks Processed: {len(inference_tasks_df)}")
print(f"Total Cumulative Reward during Inference: {total_inference_reward:.2f}")
print(f"Total Tasks Dropped during Inference: {inference_dropped_tasks}")
print(f"Total Targets Locked during Inference: {inference_locked_targets}")

# Save inference results to a CSV
inference_df = pd.DataFrame(inference_results_log)
inference_output_csv = "dqn_inference_detailed_results.csv"
inference_df.to_csv(inference_output_csv, index=False)
print(f"Detailed inference results saved to {inference_output_csv}")

# Optionally, save a summary of the inference run
with open(EVAL_LOG_FILE, "w") as f_eval_out:
    f_eval_out.write("--- DQN Inference Summary ---\n\n")
    f_eval_out.write(f"Model Path: {MODEL_PATH}\n")
    f_eval_out.write(f"Inference Tasks from: dynamic_radar_tasks_full_progression.csv ({len(inference_tasks_df)} tasks)\n")
    f_eval_out.write(f"Total Cumulative Reward: {total_inference_reward:.2f}\n")
    f_eval_out.write(f"Total Tasks Dropped: {inference_dropped_tasks}\n")
    f_eval_out.write(f"Total Targets Locked: {inference_locked_targets}\n")
    f_eval_out.write(f"Detailed results available in: {inference_output_csv}\n")
print(f"Inference summary saved to {EVAL_LOG_FILE}")


--- Starting Inference Phase ---
Model loaded successfully from dqn_radar_scheduler_dynamic_event_driven_model
Loaded 800 tasks for inference.
Processed 100/800 inference tasks. Last Reward: -0.10
Processed 200/800 inference tasks. Last Reward: -0.10
Processed 300/800 inference tasks. Last Reward: -0.10
Processed 400/800 inference tasks. Last Reward: -0.10
Processed 500/800 inference tasks. Last Reward: -0.10
Processed 600/800 inference tasks. Last Reward: -0.10
Processed 700/800 inference tasks. Last Reward: -0.10
Processed 800/800 inference tasks. Last Reward: -0.10

--- Inference Complete ---
Total Inference Tasks Processed: 800
Total Cumulative Reward during Inference: -80.00
Total Tasks Dropped during Inference: 0
Total Targets Locked during Inference: 0
Detailed inference results saved to dqn_inference_detailed_results.csv
Inference summary saved to dqn_inference_results.txt
